In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

2024-03-05 18:13:50.725754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 18:13:50.725845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 18:13:50.872391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# directory path
train_dir = '/kaggle/input/garbage-classification/Garbage classification/Garbage classification'
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255., 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)


train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


## DenseNet

In [3]:
# Loading DenseNet121 Base
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing all layers
for layer in densenet_model.layers:
    layer.trainable = False

# Adding custom classification head on top of the DenseNet121 base
x = GlobalAveragePooling2D()(densenet_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)

model = Model(inputs=densenet_model.input, outputs=predictions)

29084464/29084464 [==============================] - 0s 0us/step


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [5]:
#Creating a Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('val_accuracy') is not None:
            if logs.get('accuracy') > 0.92 and logs.get('val_accuracy') > 0.81:
                print("\nReached 90% training accuracy and 80% validation accuracy, stopping training!")
                self.model.stop_training = True

callbacks = myCallback()

In [6]:
#Compiling the Model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[callbacks]  
)

Epoch 1/50


I0000 00:00:1709662461.224205      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


126/126 [==============================] - 64s 421ms/step - loss: 1.0477 - accuracy: 0.6340 - val_loss: 0.7346 - val_accuracy: 0.7097
Epoch 2/50
126/126 [==============================] - 35s 275ms/step - loss: 0.6937 - accuracy: 0.7495 - val_loss: 0.7417 - val_accuracy: 0.7298
Epoch 3/50
126/126 [==============================] - 35s 276ms/step - loss: 0.6388 - accuracy: 0.7719 - val_loss: 0.8881 - val_accuracy: 0.6532
Epoch 4/50
126/126 [==============================] - 35s 277ms/step - loss: 0.5420 - accuracy: 0.7998 - val_loss: 0.6489 - val_accuracy: 0.7560
Epoch 5/50
126/126 [==============================] - 35s 274ms/step - loss: 0.5274 - accuracy: 0.8003 - val_loss: 0.6951 - val_accuracy: 0.7500
Epoch 6/50
126/126 [==============================] - 34s 273ms/step - loss: 0.5020 - accuracy: 0.8137 - val_loss: 0.7669 - val_accuracy: 0.7319
Epoch 7/50
126/126 [==============================] - 34s 270ms/step - loss: 0.4667 - accuracy: 0.8302 - val_loss: 0.7244 - val_accuracy: 0.7

In [8]:
import pickle
model.save_weights('densenet.weights.h5')

In [9]:
model.save('densenet_model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from IPython.display import FileLink 

In [11]:
FileLink(r'densenet.weights.h5')

/kaggle/working/densenet.weights.h5

In [12]:
FileLink(r'densenet_model.h5')

/kaggle/working/densenet_model.h5